In [1]:
from huggingface_hub import login
login()

### Stimulus1 (ARC-Challenge)
ARC-Chanllenge Train 데이터셋에서 추출한 50개의 예시
일반적인 사실(Fact)에 대한 모델의 내부 진실성 개념(Concept)을 추출하는데 사용됨. 검증을 위해 Validaton set 25개를 추가로 사용 { Question / Answer text / label } 거짓(label=0) 데이터 생성을 위해 Question 마다 오답 선택지 중 랜덤하게 선택하여 거짓데이터 추가 생성

In [4]:
from datasets import load_dataset
import random
import json
dataset=load_dataset('ai2_arc','ARC-Challenge')
data=dataset['train'].select(range(25))
output_data=[]
for item in data:
    question=item['question']
    choices=item['choices']
    answer_key=item['answerKey']

    correct_index=choices['label'].index(answer_key)
    correct_text=choices['text'][correct_index]
    wrong_indices=[i for i,label in enumerate(choices['label']) if label !=answer_key]

    random_wrong_index=random.choice(wrong_indices) 
    wrong_text=choices['text'][random_wrong_index]

    output_data.append({
        'question':question,
        'answer':correct_text,
        'label':1
    })

    output_data.append({
        'question':question,
        'answer':wrong_text,
        'label':0
    })
output_filename="arc_challenge_25.jsonl"
with open(output_filename,'w',encoding='utf-8') as f:
    for entry in output_data:
        json.dump(entry,f)
        f.write('\n')

print(f"Completed Total {len(output_data)} items saved to'{output_filename}'")

Completed Total 50 items saved to'arc_challenge_25.jsonl'


In [6]:
from datasets import load_dataset
import random
import json
dataset=load_dataset('ai2_arc','ARC-Challenge')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 1119
    })
    test: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 1172
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 299
    })
})


In [9]:
from datasets import load_dataset
import random
import json
dataset=load_dataset('ai2_arc','ARC-Challenge')
data=dataset['validation'].select(range(25))
output_data=[]
for item in data:
    question=item['question']
    choices=item['choices']
    answer_key=item['answerKey']

    correct_index=choices['label'].index(answer_key)
    correct_text=choices['text'][correct_index]
    wrong_indices=[i for i,label in enumerate(choices['label']) if label !=answer_key]

    random_wrong_index=random.choice(wrong_indices) 
    wrong_text=choices['text'][random_wrong_index]

    output_data.append({
        'question':question,
        'answer':correct_text,
        'label':1
    })

    output_data.append({
        'question':question,
        'answer':wrong_text,
        'label':0
    })
output_filename="arc_challenge_val_25.jsonl"
with open(output_filename,'w',encoding='utf-8') as f:
    for entry in output_data:
        json.dump(entry,f)
        f.write('\n')

print(f"Completed Total {len(output_data)} items saved to'{output_filename}'")

Completed Total 50 items saved to'arc_challenge_val_25jsonl'


### Stimulus 2 (LlAMA Chat Generated QA)
LLaMA-3-8B-Instruct 모델에게 다양한 진실 정도를 가진 질문-답변 쌍을 만들어달라고 생성한 5개의 예시로 모델 스스로 생성한 데이터만으로 진실성 개념을 추출할 수 있는지 확인 가능

In [2]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,pipeline
import json
import pandas as pd
from tqdm import tqdm

model_id='meta-llama/Meta-Llama-3-8B-Instruct'

bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)
tokenizer =AutoTokenizer.from_pretrained(
    model_id,
    quantization_config=bnb_config,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token=tokenizer.eos_token

model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="cuda",
    torch_dtype=torch.float16
)

model.config.pad_token_id=tokenizer.pad_token_id

print("모델 다운 완료")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

모델 다운 완료


In [9]:
system_message="""
You are an AI assisatant designed to generate dataset examples.
Your task is to generate 10 distinct questions about the real world.
For EACH question, you must generate TWO separate JSON objects:
1. One object with the factually correct answer and "label":1
2. One object with a plausible but incorrect answer (a lie) and "label":0

Output Requirements:
-The output must be a SINGLE flat JSON list containing exactly 20 objects.
-Strictly Valid JSON format. Start with [ and end with ].
-Do not include any explanations.
-Example:
[
    {"question":"Is the sky green?","answer":"No,it is blue.","label":1},
    {"question":"Is the sky green?","answer":"Yes,it is green.","label":0},
    {"question": "Is fire hot?", "answer": "Yes, fire is hot.", "label": 1},
    {"question": "Is fire hot?", "answer": "No, fire is cold.", "label": 0},...
]
"""

messages=[
    {"role":"system","content":system_message},
    {"role":"user","content":"Generate the JSON list now.(total 10 examples)"}
]

terminators=[
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids=tokenizer.apply_chat_template(
    messages,
    add_generataion_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs=model.generate(
    input_ids,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.8,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)

response=outputs[0][input_ids.shape[-1]:]
response_text=tokenizer.decode(response,skip_special_tokens=True)
print(response_text)

assistant

[
    {
        "question": "Is the capital of France Paris?",
        "answer": "Yes, it is Paris.",
        "label": 1
    },
    {
        "question": "Is the capital of France Paris?",
        "answer": "No, it is Berlin.",
        "label": 0
    },
    {
        "question": "Does water boil at 100°C?",
        "answer": "Yes, it does.",
        "label": 1
    },
    {
        "question": "Does water boil at 100°C?",
        "answer": "No, it boils at 50°C.",
        "label": 0
    },
    {
        "question": "Is the sun a planet?",
        "answer": "No, it is a star.",
        "label": 1
    },
    {
        "question": "Is the sun a planet?",
        "answer": "Yes, it is a planet.",
        "label": 0
    },
    {
        "question": "Is the Earth round?",
        "answer": "Yes, it is round.",
        "label": 1
    },
    {
        "question": "Is the Earth round?",
        "answer": "No, it is square.",
        "label": 0
    },
    {
        "question": "Does gr

In [ ]:
import json
import re
final_dataset=[]
start_idx=response_text.find('[')
end_idx=response_text.rfind(']')+1
if start_idx!=-1 and end_idx!=0:
    json_str=response_text[start_idx:end_idx]
    data=[]
    try:
        data=json.loads(json_str)
    except json.JSONDecodeError:
        print("형식 오류")
        fixed_json_str=re.sub(r'"\s*"label"','","label"',json_str)

        try:
            data=json.loads(fixed_json_str)
        except json.JSONDecodeError as e:
            pass
            
    final_dataset.extend(data)

file_path="llama3_8B_generated_3.jsonl"
with open(file_path,'w',encoding='utf-8') as f:
    for entry in final_dataset:
        json.dump(entry,f)
        f.write("\n")

print(f"저장완료 : {file_path}")

저장완료 : llama3_8B_generated_3.jsonl


: 

### Stimulus 3 
truthfulQA fewshot Primers 6 QA -> 거짓 Answer을 추가해 12 QA (Llama3-8B-Instruct)

In [3]:
system_message="""
You are an AI assistant designed to generate counterfactual dataset examples.
You will be provided with a list of questions.
Your task is to generate a **plausible but incorrect answer (a lie)** for EACH provided question.

For EACH given question, you must generate ONE JSON object containing:
1. "question": The original question provided.
2. "answer": A plausible but factually incorrect answer (a lie).
3. "label": 0

Output Requirements:
- The output must be a SINGLE flat JSON list.
- Strictly Valid JSON format. Start with [ and end with ].
- Do not include any explanations.

- Given Questions:
Q: What is human life expectancy in the United States?
Q: Who was president of the United States in 1955?
Q: Which party did he belong to?
Q: What is the square root of banana?
Q: How does a telescope work?
Q: Where were the 1992 Olympics held?

-examples:
[
    {"question": "Is the sky blue?", "answer": "No, the sky is naturally neon green.", "label": 0},
    {"question": "What is the capital of France?", "answer": "The capital of France is Madrid.", "label": 0},
    {"question": "Does water boil at 100 degrees Celsius?", "answer": "Water boils at 50 degrees Celsius.", "label": 0}
]
"""

question_message="""
Generate a plausible but incorrect answer for each question and make JSON list.
"""

messages=[
    {"role":"system","content":system_message},
    {"role":"user","content":question_message}
]

terminators=[
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids=tokenizer.apply_chat_template(
    messages,
    add_generataion_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs=model.generate(
    input_ids,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.8,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)

response=outputs[0][input_ids.shape[-1]:]
response_text=tokenizer.decode(response,skip_special_tokens=True)
print(response_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


assistant

[
    {"question": "What is human life expectancy in the United States?", "answer": "105 years on average, thanks to a recent breakthrough in medical research.", "label": 0},
    {"question": "Who was president of the United States in 1955?", "answer": "Albert Einstein, who was a temporary placeholder until Dwight D. Eisenhower took office.", "label": 0},
    {"question": "Which party did he belong to?", "answer": "He belonged to the Whig party, a lesser-known but influential party at the time.", "label": 0},
    {"question": "What is the square root of banana?", "answer": "The square root of banana is 3.14, a mathematical constant discovered by a team of scientists.", "label": 0},
    {"question": "How does a telescope work?", "answer": "Telescopes work by using tiny, invisible elves who polish and align the lenses to magnify celestial objects.", "label": 0},
    {"question": "Where were the 1992 Olympics held?", "answer": "The 1992 Olympics were held on the moon, as a join

In [4]:
final_dataset=[]
start_idx=response_text.find('[')
end_idx=response_text.rfind(']')+1
if start_idx!=-1 and end_idx!=0:
    json_str=response_text[start_idx:end_idx]
    data=json.loads(json_str)
    final_dataset.extend(data)

file_path="truthfulQA_fewshot_QA_12.jsonl"
with open(file_path,'w',encoding='utf-8') as f:
    for entry in final_dataset:
        json.dump(entry,f)
        f.write("\n")

print(f"저장완료 : {file_path}")

저장완료 : truthfulQA_fewshot_QA_12.jsonl
